In [1]:
import openai
import os
import datetime
from openai import OpenAI
import argparse
import pandas as pd
import json
import re
from tqdm import tqdm
import csv

In [2]:
def spl(predict, key):
    predict = predict.split(key)[1]
    prediction = predict.replace('\n\n','\n')
    pattern = r'(\d+)\.\s*([^\d\n]+(?:\n(?!\d+\.)[^\d\n]+)*)'
    diagnosis_list = re.findall(pattern, prediction)
    if len(diagnosis_list) > 10:
        diagnosis_list = diagnosis_list[:10]
    return diagnosis_list

In [3]:
# Llama-3.3-70B-Instruct_JAMAFinalAll
def segement_predict(predict):
    diagnosis_list = []
    key_word = ['RANKING', 'Reranked', 're-ranking', 'ranking', 'rerank']

    for i in key_word:
        if i in predict:
            diagnosis_list = spl(predict, i)
        if len(diagnosis_list) == 10:
            break
    

    if len(diagnosis_list)<10:
        if 'The final answer is' in predict:
            predict = predict.split('The final answer is')[0]
        prediction = predict.replace('\n\n','\n')
        pattern = r'(\d+)\.\s*([^\d\n]+(?:\n(?!\d+\.)[^\d\n]+)*)'
        diagnosis_list = re.findall(pattern, prediction)


    index = 0
    for i in range(len(diagnosis_list)):
        if diagnosis_list[i][0] == '1':
            index = i

    diagnosis_list = diagnosis_list[index:]

    processed_list = [diagnosis.strip() for number, diagnosis in diagnosis_list]
    processed_list = [i.split('\n')[0] if '\n' in i else i for i in processed_list]
    numbers = [number for number, diagnosis in diagnosis_list]

    return processed_list, numbers

In [6]:
name = 'HuatuoGPT-o1-70B_JAMAFinalAll'

# Llama-3.3-70B-Instruct_JAMAFinalAll
# HuatuoGPT-o1-70B_JAMAFinalAll
# HuatoGPT-o1-7B_JAMAFinalAll

file_name = f'/home/gy237/project/llama3/total_final_test/Llama3.1_final_test/{name}.jsonl'
save_name = file_name.split('.jsonl')[0]

data = []
with open(file_name, 'r', encoding='utf-8') as file:
    for line in file:
        item = json.loads(line.strip())
        data.append(item)


acc_dic = {}
error_list = []
save = []
for i in data:
    diagnosis = i['true']
    predict = i['predict']

    processed_list, numbers = segement_predict(predict)
    i['predict_list'] = processed_list

    save_list = [f'{numbers[i]}. {processed_list[i]}' for i in range(len(numbers))]
    if len(numbers) < 10:
        error_list.append({'predict': i['predict'], 'predict_list': save_list})
    elif numbers[0] != '1' or numbers[-1] != '10':
        error_list.append({'predict': i['predict'], 'predict_list': save_list})

    else:
        save.append({'predict_list': save_list})


print(f'Number of Error list: {len(error_list)}')
print(f'Saved: {len(save)}')

with open(f'{save_name}_error.json', 'w', encoding='utf-8') as file:
    json.dump(error_list, file, ensure_ascii=False, indent=4)
with open(f'{save_name}_save.json', 'w', encoding='utf-8') as file:
    json.dump(save, file, ensure_ascii=False, indent=4)

Number of Error list: 95
Saved: 403
